In [35]:
import pandas as pd
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [36]:
df1 = pd.read_csv("data/labeled01.csv", encoding='cp1252')
df2 = pd.read_csv("data/summaries_t5.csv")
df3 = pd.read_csv("data/summaries_bart.csv")
df4 = pd.read_csv("data/summaries_pgs.csv")
df = pd.DataFrame()
df['human_summary'] = df1['summary'][0:150]
df['t5_summary'] = df2['summary'][0:150]
df['bart_summary'] = df3['summary'][0:150]
df['pgs_summary'] = df4['summary'][0:150]


In [37]:
df.head(2)

,human_summary,t5_summary,bart_summary,pgs_summary
0,The Miami City Commission accepted bids from r...,miami city commission accepts bids for bio haz...,"Bids received on february 18, 2021 pursuant to...","City commission meeting agenda January 13, 202..."
1,The Miami City Commission found that the COVID...,miami city commission finds coronavirus 2019 p...,The city commission found that the novel coron...,The commission finds that the novel coronaviru...


In [38]:
rouge = Rouge()
total_rouge_1 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_2 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_l = {'r': 0, 'p': 0, 'f': 0}
num_rows = len(df)

for _, row in df.iterrows():
    rouge_scores = rouge.get_scores(row['t5_summary'], row['human_summary'])[0]
    for metric in ['rouge-1', 'rouge-2', 'rouge-l']:
        for score_type in ['r', 'p', 'f']:
            if metric == 'rouge-1':
                total_rouge_1[score_type] += rouge_scores[metric][score_type]
            elif metric == 'rouge-2':
                total_rouge_2[score_type] += rouge_scores[metric][score_type]
            else:
                total_rouge_l[score_type] += rouge_scores[metric][score_type]

avg_rouge_1 = {score_type: total_rouge_1[score_type] / num_rows for score_type in ['r', 'p', 'f']}
avg_rouge_2 = {score_type: total_rouge_2[score_type] / num_rows for score_type in ['r', 'p', 'f']}
avg_rouge_l = {score_type: total_rouge_l[score_type] / num_rows for score_type in ['r', 'p', 'f']}


from nltk.translate.bleu_score import sentence_bleu

df['bleu_score'] = df.apply(lambda row: sentence_bleu([row['human_summary'].split()], row['t5_summary'].split()), axis=1)
avg_bleu_score = df['bleu_score'].mean()


print(f'Average ROUGE-1: {avg_rouge_1}')
print(f'Average ROUGE-2: {avg_rouge_2}')
print(f'Average ROUGE-L: {avg_rouge_l}')
print(f'Average BLEU score: {avg_bleu_score:.4f}')

Average ROUGE-1: {'r': 0.34971456818519153, 'p': 0.2824412725619341, 'f': 0.3027437794904844}
Average ROUGE-2: {'r': 0.17817113763707526, 'p': 0.1285662444559896, 'f': 0.14427594367596716}
Average ROUGE-L: {'r': 0.33015373136225873, 'p': 0.2649642571221646, 'f': 0.2847871839169802}
Average BLEU score: 0.0698


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many

In [39]:
# rouge = Rouge()
# rouge_scores_bart = [rouge.get_scores(row['bart_summary'], row['human_summary']) for _, row in df.iterrows()]

# # prepare data for bleu calculation
# nlp_summaries_lists = [[summary.split()] for summary in df['bart_summary']]
# human_summaries_lists = [summary.split() for summary in df['human_summary']]
# bleu_score_bart = corpus_bleu(nlp_summaries_lists, human_summaries_lists)

# print('ROUGE scores:', rouge_scores_bart)
# print('BLEU score:', bleu_score_bart)


rouge = Rouge()
total_rouge_1 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_2 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_l = {'r': 0, 'p': 0, 'f': 0}
num_rows = len(df)

for _, row in df.iterrows():
    rouge_scores = rouge.get_scores(row['bart_summary'], row['human_summary'])[0]
    for metric in ['rouge-1', 'rouge-2', 'rouge-l']:
        for score_type in ['r', 'p', 'f']:
            if metric == 'rouge-1':
                total_rouge_1[score_type] += rouge_scores[metric][score_type]
            elif metric == 'rouge-2':
                total_rouge_2[score_type] += rouge_scores[metric][score_type]
            else:
                total_rouge_l[score_type] += rouge_scores[metric][score_type]

avg_rouge_1 = {score_type: total_rouge_1[score_type] / num_rows for score_type in ['r', 'p', 'f']}
avg_rouge_2 = {score_type: total_rouge_2[score_type] / num_rows for score_type in ['r', 'p', 'f']}
avg_rouge_l = {score_type: total_rouge_l[score_type] / num_rows for score_type in ['r', 'p', 'f']}


from nltk.translate.bleu_score import sentence_bleu

df['bleu_score'] = df.apply(lambda row: sentence_bleu([row['human_summary'].split()], row['bart_summary'].split()), axis=1)
avg_bleu_score = df['bleu_score'].mean()


print(f'Average ROUGE-1: {avg_rouge_1}')
print(f'Average ROUGE-2: {avg_rouge_2}')
print(f'Average ROUGE-L: {avg_rouge_l}')
print(f'Average BLEU score: {avg_bleu_score:.4f}')

Average ROUGE-1: {'r': 0.39058341843081357, 'p': 0.2951865067673829, 'f': 0.3269490769023268}
Average ROUGE-2: {'r': 0.20322398369646003, 'p': 0.14357114338755977, 'f': 0.1621193815429138}
Average ROUGE-L: {'r': 0.3601535004659526, 'p': 0.27199057785140773, 'f': 0.30123608929090095}
Average BLEU score: 0.0924


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many

In [40]:
# rouge = Rouge()
# rouge_scores_pgs = [rouge.get_scores(row['pgs_summary'], row['human_summary']) for _, row in df.iterrows()]

# # prepare data for bleu calculation
# nlp_summaries_lists = [[summary.split()] for summary in df['pgs_summary']]
# human_summaries_lists = [summary.split() for summary in df['human_summary']]
# bleu_score_pgs = corpus_bleu(nlp_summaries_lists, human_summaries_lists)

# print('ROUGE scores:', rouge_scores_pgs)
# print('BLEU score:', bleu_score_pgs)


rouge = Rouge()
total_rouge_1 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_2 = {'r': 0, 'p': 0, 'f': 0}
total_rouge_l = {'r': 0, 'p': 0, 'f': 0}
num_rows = len(df)

for _, row in df.iterrows():
    rouge_scores = rouge.get_scores(row['pgs_summary'], row['human_summary'])[0]
    for metric in ['rouge-1', 'rouge-2', 'rouge-l']:
        for score_type in ['r', 'p', 'f']:
            if metric == 'rouge-1':
                total_rouge_1[score_type] += rouge_scores[metric][score_type]
            elif metric == 'rouge-2':
                total_rouge_2[score_type] += rouge_scores[metric][score_type]
            else:
                total_rouge_l[score_type] += rouge_scores[metric][score_type]

avg_rouge_1 = {score_type: total_rouge_1[score_type] / num_rows for score_type in ['r', 'p', 'f']}
avg_rouge_2 = {score_type: total_rouge_2[score_type] / num_rows for score_type in ['r', 'p', 'f']}
avg_rouge_l = {score_type: total_rouge_l[score_type] / num_rows for score_type in ['r', 'p', 'f']}


from nltk.translate.bleu_score import sentence_bleu

df['bleu_score'] = df.apply(lambda row: sentence_bleu([row['human_summary'].split()], row['pgs_summary'].split()), axis=1)
avg_bleu_score = df['bleu_score'].mean()


print(f'Average ROUGE-1: {avg_rouge_1}')
print(f'Average ROUGE-2: {avg_rouge_2}')
print(f'Average ROUGE-L: {avg_rouge_l}')
print(f'Average BLEU score: {avg_bleu_score:.4f}')

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Average ROUGE-1: {'r': 0.3764299453576587, 'p': 0.2627328420204445, 'f': 0.3019300683261253}
Average ROUGE-2: {'r': 0.184709117630587, 'p': 0.11991324221889106, 'f': 0.1411673781828742}
Average ROUGE-L: {'r': 0.34935435693012534, 'p': 0.2434059862425279, 'f': 0.2799091869075838}
Average BLEU score: 0.0757


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [41]:
# Load the trained model
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import Trainer, TrainingArguments
model = BartForConditionalGeneration.from_pretrained("./results2/checkpoint-500")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Define a function to generate summaries
def generate_summary(text, model, tokenizer):
    # Tokenize the input text
    input_ids = tokenizer.encode(text, return_tensors='pt')
    
    # Generate the summary
    summary_ids = model.generate(input_ids, do_sample=True, max_new_tokens=50, min_length=10)
    
    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary

# # Example usage
# text = "resolution miami city commission accepting bids received february pursuant invitation bid responsive responsible bidders bioresponse corp florida profit corporation jrp sons corporation fl cleanup florida profit corporation muma sa lc florida limited liability company ifb group pre qualified pool vendors provision bio hazardous waste spills decontamination services needed basis citywide initial term two years option renew two additional two year periods allocating funds end user departments sources funds subject availability funds budgetary approval time need authorizing city manager negotiate execute documents including amendments renewals extensions subject allocations appropriations prior budgetary approvals compliance applicable provisions code city miami florida amended including city procurement ordinance anti deficiency act financial integrity principles set forth chapter city code forms acceptable city attorney compliance applicable laws rules regulations may deemed necessary said purpose city commission meeting agenda january city miami page printed ca department real estate asset management resolution."
# summary = generate_summary(text, model, tokenizer)
# print(summary)

In [42]:
# stop_words = set(stopwords.words('english')) 
def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text #remove links
    # newString = re.sub(r'\([^)]*\)', '', newString) # remove text in the paranthesis
    newString = re.sub('_','', newString) # removing underscores
    newString = re.sub('"','', newString) # removing double quotes
    newString = re.sub('/',' ', newString) # removing double quotes
    newString = re.sub('-','', newString) # removing double quotes
    newString = re.sub(r'[^\x00-\x7F]+', '', newString) # removes non-ASCII characters
    # newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString) #remove possesive s
    # newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    # if(num==0):
    #     tokens = [w for w in newString.split() if not w in stop_words]
    # else:
    #     tokens=newString.split()
    tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [43]:
df_test = df1.iloc[0:26]
df_test.head()

,resolution,summary
0,resolution of the miami city commission accept...,The Miami City Commission accepted bids from r...
1,resolution of the miami city commission findin...,The Miami City Commission found that the COVID...
2,resolution of the miami city commission author...,Miami City Commission authorizes city manager ...
3,"resolution of the miami city commission, with ...",Miami City Commission accepts perpetual sidewa...
4,"resolution of the miami city commission, with ...",Miami City Commission accepts two right-of-way...


In [44]:
# Create an empty list to store the summaries
summaries = []

for text in df_test['resolution']:
    summary = generate_summary(text_cleaner(text,0), model, tokenizer)
    summaries.append(summary)

# Create a new DataFrame with the summaries
summary_df = pd.DataFrame({'summary': summaries})

# Save the DataFrame to a CSV file
# summary_df.to_csv('summaries.csv', index=False)

In [45]:
summary_df.head(2)

,summary
0,The Miami city commission accepts bids receive...
1,The Miami city commission finds that the novel...


In [46]:
df['bart_n_summary'] = summary_df['summary']

In [47]:
df.head(2)

,human_summary,t5_summary,bart_summary,pgs_summary,bleu_score,bart_n_summary
0,The Miami City Commission accepted bids from r...,miami city commission accepts bids for bio haz...,"Bids received on february 18, 2021 pursuant to...","City commission meeting agenda January 13, 202...",1.201524e-231,The Miami city commission accepts bids receive...
1,The Miami City Commission found that the COVID...,miami city commission finds coronavirus 2019 p...,The city commission found that the novel coron...,The commission finds that the novel coronaviru...,4.955625e-155,The Miami city commission finds that the novel...


In [48]:
rouge = Rouge()
rouge_scores_pgs = [rouge.get_scores(row['bart_n_summary'], row['human_summary']) for _, row in df.iterrows()]

# prepare data for bleu calculation
nlp_summaries_lists = [[summary.split()] for summary in df['bart_n_summary']]
human_summaries_lists = [summary.split() for summary in df['human_summary']]
bleu_score_pgs = corpus_bleu(nlp_summaries_lists, human_summaries_lists)

print('ROUGE scores:', rouge_scores_pgs)
print('BLEU score:', bleu_score_pgs)

AssertionError: 